In [20]:
import os
import os.path as osp
import glob

import torch
import numpy as np
import cv2
from patchify import patchify
from sklearn.model_selection import train_test_split

## Define Preprocessing Functions

In [21]:
def apply_morphology(mask, radius_erosion=1, iter_erosion=1, radius_dilation=3, iter_dilation=2, morph_op="opening"):
    """Applies erosion and dilation to the mask."""
    kernel_erosion = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (radius_erosion, radius_erosion))
    kernel_dilation = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (radius_dilation, radius_dilation))

    if morph_op == "opening":
        temp = cv2.erode(mask, kernel_erosion, iterations=iter_erosion)
        output = cv2.dilate(temp, kernel_dilation, iterations=iter_dilation)
    elif morph_op == "closing":
        temp = cv2.dilate(mask, kernel_dilation, iterations=iter_dilation)
        output = cv2.erode(temp, kernel_erosion, iterations=iter_erosion)
    else:
        raise ValueError("Invalid operation. Choose 'opening' or 'closing'.")
    return output

def get_bounding_box(mask):
    """Extracts bounding box from a mask."""
    y_indices, x_indices = np.where(mask > 0)

    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)

    # add perturbation to box coordinates
    H, W = mask.shape
    x_min = max(0, x_min - np.random.randint(0, 20))
    x_max = min(W, x_max + np.random.randint(0, 20))
    y_min = max(0, y_min - np.random.randint(0, 20))
    y_max = min(H, y_max + np.random.randint(0, 20))

    return np.array([x_min, y_min, x_max, y_max])

## Variables

In [22]:
# Data directories
DATA_ROOT = "captures"
IMAGE_SUBDIR = "raw"
MASK_SUBDIR = "masked"

SAVE_ROOT = "processed_data"
IMAGE_SAVE_SUBDIR = "images"
MASK_SAVE_SUBDIR = "masks"
BOX_SAVE_SUBDIR = "boxes"

# Preprocessing parameters
# TODO: Add
PATCH_SIZE = 256
PATCH_STEP = 256
radius_erosion = 1
iter_erosion = 1
radius_dilation = 3
iter_dilation = 2



## Preprocess Data

The following cell iterates through DATA_ROOT/ and preprocess the according image-mask pairs. Then it saves the processed data under SAVE_ROOT/ .
For further implementational convinience, a few notations were changed and file naming has changed to `{id}_p{patch_id}.png`

In [ ]:
image_paths = glob.glob(osp.join(DATA_ROOT, IMAGE_SUBDIR, "*.png"))
for image_path in image_paths:
    mask_path = image_path.replace(IMAGE_SUBDIR, MASK_SUBDIR)
    if not osp.exists(mask_path):
        print(f"Mask not found for {image_path}")
        continue
    image = cv2.imread(image_path)[:, :, :3]  # Ensure RGB
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Apply erosion/dilation to the mask
    mask = apply_morphology(mask)

    # Patchify
    img_patches = patchify(image, (PATCH_SIZE, PATCH_SIZE, 3), step=PATCH_STEP) #(numH, numW, 1, patchH, patchW, C)
    mask_patches = patchify(mask, (PATCH_SIZE, PATCH_SIZE), step=PATCH_STEP) #(numH, numW, patchH, patchW)

    patch_cnt = 0
    for i in range(img_patches.shape[0]):
        for j in range(img_patches.shape[1]):
            patch_img = img_patches[i, j, 0, :, :, :]
            patch_mask = mask_patches[i, j, :, :]

            # Skip empty patches
            if np.max(patch_mask) == 0:
                continue

            # Get bounding Box
            box = get_bounding_box(patch_mask)

            # Save patch images and masks in 'process_data' as 00001_p01.png (1st image, 1st patch )
            img_basename = int(osp.basename(image_path).replace(".png", "").replace("raw_", ""))
            patch_img_save_path = osp.join(SAVE_ROOT, IMAGE_SAVE_SUBDIR, f"{img_basename:05d}_p{patch_cnt:01d}.png")
            patch_mask_save_path = osp.join(SAVE_ROOT, MASK_SAVE_SUBDIR, f"{img_basename:05d}_p{patch_cnt:01d}.png")
            patch_box_save_key  = osp.join(SAVE_ROOT, BOX_SAVE_SUBDIR, f"{img_basename:05d}_p{patch_cnt:01d}.npy")

            # Ensure directories exist
            os.makedirs(osp.dirname(patch_img_save_path), exist_ok=True)
            os.makedirs(osp.dirname(patch_mask_save_path), exist_ok=True)
            os.makedirs(osp.dirname(patch_box_save_path), exist_ok=True)

            # Save the patches/boxes
            cv2.imwrite(patch_img_save_path, patch_img)
            cv2.imwrite(patch_mask_save_path, patch_mask)
            np.save(patch_box_save_path, box)
      
            patch_cnt += 1
    np.savez(path)

In [40]:
print(np.load("processed_data/boxes/00002_p0.npz").file)

AttributeError: 'NpzFile' object has no attribute 'file'